## import modules

In [1]:
from tqdm import tqdm
import re
import torch
import os
import random
import json
import numpy as np

import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer

In [2]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  # 디버깅 위한 세팅
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

print('Device:', torch.cuda.device)  # 출력결과: cuda 
print('Count of using GPUs:', torch.cuda.device_count())   
print('Current cuda device:', torch.cuda.current_device()) 

Device: <class 'torch.cuda.device'>
Count of using GPUs: 2
Current cuda device: 0


### Load Data
Data가 위치한 PATH에서 Data를 불러오기

In [3]:
def jsonload(fname, encoding="utf-8"):
    with open(fname, encoding=encoding) as f:
        j = json.load(f)

    return j

# json 개체를 파일이름으로 깔끔하게 저장
def jsondump(j, fname):
    with open(fname, "w", encoding="UTF8") as f:
        json.dump(j, f, ensure_ascii=False)

# jsonl 파일 읽어서 list에 저장
def jsonlload(fname, encoding="utf-8"):
    json_list = []
    with open(fname, encoding=encoding) as f:
        for line in f.readlines():
            json_list.append(json.loads(line))
    return json_list

In [4]:
train_data = jsonlload('nikluge-sa-2022-train.jsonl')
dev_data = jsonlload('nikluge-sa-2022-dev.jsonl')

## Preprocessing method

In [5]:
def preprocess(text: str, only_kor: bool = True):
    """한국어 문장을 옵션에 맞게 전처리"""
    # 한국어 모음과 특수 문자, 숫자 및 영어 제거
    if only_kor:
        text = re.sub(f"[^가-힣| |]+", "", text)
    else:
        text = re.sub(f"[^가-힣|ㄱ-ㅎ|0-9|]+", "", text)

    # 연속 공백 제거
    text = re.sub(" +", " ", text)

    # 좌우 불필요한 공백 제거
    return text.strip()

## 재현성 위한 seed 고정

In [6]:
def seed_everything(seed:int = 1004):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
seed_everything(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## 태그셋 정의

In [7]:
#개체#속성 태그셋 정의
entity_property_pair = [
    '제품 전체#품질', '제품 전체#편의성', '제품 전체#디자인', '제품 전체#일반', '제품 전체#가격', 
    '제품 전체#인지도',  '제품 전체#다양성',
    '패키지/구성품#디자인', '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#일반',
    '패키지/구성품#편의성', '패키지/구성품#품질',
    '본품#일반', '본품#다양성', '본품#품질', '본품#인지도', '본품#편의성', '본품#디자인', '본품#가격',
    '브랜드#인지도', '브랜드#일반', '브랜드#디자인', '브랜드#품질', '브랜드#가격']

rep_entity_property_pair = [
    '제품전체#품질', '제품전체#편의성', '제품전체#디자인', '제품전체#일반', '제품전체#가격', 
    '제품전체#인지도',  '제품전체#다양성',
    '패키지/구성품#디자인', '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#일반',
    '패키지/구성품#편의성', '패키지/구성품#품질',
    '본품#일반', '본품#다양성', '본품#품질', '본품#인지도', '본품#편의성', '본품#디자인', '본품#가격',
    '브랜드#인지도', '브랜드#일반', '브랜드#디자인', '브랜드#품질', '브랜드#가격']

len(entity_property_pair)

25

## Load Tokenizer

In [8]:
MODEL_NAME = 'klue/roberta-large'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer

PreTrainedTokenizerFast(name_or_path='klue/roberta-large', vocab_size=32000, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [9]:
special_tokens_dict = {
    'additional_special_tokens': rep_entity_property_pair
}

print(tokenizer.vocab_size)

# 토크나이저는 각 모델 별로 만들지 않아도 된다. [임베딩 layer에만 영향 주므로]
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

print(num_added_toks)

32000
25


In [10]:
print(tokenizer.tokenize("야 아이패드 가격 왜 이래 본품#가격"))
print(tokenizer("야 아이패드 가격 왜 이래 본품#가격"))
print(tokenizer.decode(tokenizer.encode("야 아이패드 가격 왜 이래 본품#가격")))

['야', '아이패드', '가격', '왜', '이래', '본품#가격']
{'input_ids': [0, 1396, 15641, 3852, 1460, 5625, 32019, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}
[CLS] 야 아이패드 가격 왜 이래 본품#가격 [SEP]


## Load Model

In [11]:
ACD_set = []

for i in range(len(rep_entity_property_pair)):
    tmp_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)  #True/False 
    # tmp_model.resize_token_embeddings(tokenizer.vocab_size + num_added_toks)
    # ACD 모델은 태그셋이 input으로 들어가지 않으므로 resize 안함.
    ACD_set.append(tmp_model)

ACD_set[7]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifi

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifi

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

## 데이터 준비

In [12]:
polarity_id_to_name = ['positive', 'negative', 'neutral']  # 차례대로 0, 1, 2
polarity_name_to_id = {polarity_id_to_name[i]: i for i in range(len(polarity_id_to_name))}

In [13]:
ACD_Datas = [[] for i in range(len(entity_property_pair))]  # 속성 각각마다의 데이터 셋
ACD_labels = [[] for i in range(len(entity_property_pair))]

POL_Datas = []
POL_labels = []

### sentence들 먼저 모으고 한꺼번에 tokenize 하자

for idx, pair in enumerate(rep_entity_property_pair):
    for idx2, datas in enumerate(train_data):
        sen = preprocess(datas['sentence_form'])
        annos = datas['annotation']  # 여러 개 일 수도 있음. (이중 리스트)
        check_point = False

        """
        개체#속성 태그 셋의 경우 sentence를 보고 모델이 태그를 True/False로 알려줄 것이므로 pair를 넣으면 안됨.
        
        감성 분석을 위한 데이터셋의 경우 sentence + 태그를 본 후 해당 태그에 대한 감성 분석을 기대하므로 pair를 함께 넣어줌.
        """
        
        ACD_Datas[idx].append(sen)
        
        for idx3, annotation in enumerate(annos):
            entity_property = annotation[0]  # raw_data의 annotation 추출
            entity_property = entity_property.replace(" ", "")  # 띄어쓰기 제거
            polarity = annotation[2]

            if entity_property == pair:
                check_point = True
                
            else:
                ;
                
        if check_point:
            ACD_labels[idx].append(1)  # 해당 태그 맞음.
            POL_Datas.append(sen + " "+ pair)
            POL_labels.append(polarity_name_to_id[polarity])
        
        else:
            ACD_labels[idx].append(0)

In [14]:
devs = dev_data[:838]
tests = dev_data[838:]

In [15]:
################### Dev set #######################
dev_ACD_Datas = [[] for i in range(len(entity_property_pair))]  # 속성 각각마다의 데이터 셋
dev_ACD_labels = [[] for i in range(len(entity_property_pair))]

dev_POL_Datas = []
dev_POL_labels = []

### sentence들 먼저 모으고 한꺼번에 tokenize 하자

for idx, pair in enumerate(rep_entity_property_pair):
    for idx2, datas in enumerate(devs):
        sen = preprocess(datas['sentence_form'])
        annos = datas['annotation']  # 여러 개 일 수도 있음. (이중 리스트)
        check_point = False

        """
        개체#속성 태그 셋의 경우 sentence를 보고 모델이 태그를 True/False로 알려줄 것이므로 pair를 넣으면 안됨.
        
        감성 분석을 위한 데이터셋의 경우 sentence + 태그를 본 후 해당 태그에 대한 감성 분석을 기대하므로 pair를 함께 넣어줌.
        """
        
        dev_ACD_Datas[idx].append(sen)
        
        for idx3, annotation in enumerate(annos):
            entity_property = annotation[0]  # raw_data의 annotation 추출
            entity_property = entity_property.replace(" ", "")  # 띄어쓰기 제거
            polarity = annotation[2]

            if entity_property == pair:
                check_point = True
                
            else:
                ;
                
        if check_point:
            dev_ACD_labels[idx].append(1)  # 해당 태그 맞음.
            dev_POL_Datas.append(sen + " "+ pair)
            dev_POL_labels.append(polarity_name_to_id[polarity])
        
        else:
            dev_ACD_labels[idx].append(0)

In [16]:
################### Test set #######################
test_ACD_Datas = [[] for i in range(len(entity_property_pair))]  # 속성 각각마다의 데이터 셋
test_ACD_labels = [[] for i in range(len(entity_property_pair))]

test_POL_Datas = []
test_POL_labels = []

### sentence들 먼저 모으고 한꺼번에 tokenize 하자

for idx, pair in enumerate(rep_entity_property_pair):
    for idx2, datas in enumerate(tests):
        sen = preprocess(datas['sentence_form'])
        annos = datas['annotation']  # 여러 개 일 수도 있음. (이중 리스트)
        check_point = False

        """
        개체#속성 태그 셋의 경우 sentence를 보고 모델이 태그를 True/False로 알려줄 것이므로 pair를 넣으면 안됨.
        
        감성 분석을 위한 데이터셋의 경우 sentence + 태그를 본 후 해당 태그에 대한 감성 분석을 기대하므로 pair를 함께 넣어줌.
        """
        
        test_ACD_Datas[idx].append(sen)
        
        for idx3, annotation in enumerate(annos):
            entity_property = annotation[0]  # raw_data의 annotation 추출
            entity_property = entity_property.replace(" ", "")  # 띄어쓰기 제거
            polarity = annotation[2]

            if entity_property == pair:
                check_point = True
                
            else:
                ;
                
        if check_point:
            test_ACD_labels[idx].append(1)  # 해당 태그 맞음.
            test_POL_Datas.append(sen + " "+ pair)
            test_POL_labels.append(polarity_name_to_id[polarity])
        
        else:
            test_ACD_labels[idx].append(0)

In [17]:
class klue_Dataset(torch.utils.data.Dataset):
    def __init__(self, dataset, label):  # 전처리된 데이터 셋이 들어옴
        self.dataset = dataset
        self.label = label

    def __getitem__(self, idx):
        # getitem이므로 gradient 계산에 영향을 주지 않게 clone().detach() 실행
        item = {key: val[idx].clone().detach() for key, val in self.dataset.items()}
        item['label'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):  # 샘플 수
        return len(self.label)

In [18]:
klue_sets = []

for i in range(len(rep_entity_property_pair)):
    tok_sen = tokenizer(ACD_Datas[i], padding='max_length', return_tensors="pt",
                    max_length=256, truncation=True, add_special_tokens=True)  
    
    klue_sets.append(klue_Dataset(tok_sen, ACD_labels[i]))  # klue_Dataset에서 1차원 텐서로 바뀜.

In [19]:
klue_sets[24].__getitem__(758)

{'input_ids': tensor([    0,  8155, 14470,  6653, 18319,   886,  5046,  2047,  2377,  2170,
          7198,  2470,  7134,  2031,  2299,  2118, 20082,  2299,  2118,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [20]:
dev_klue_sets = []

for i in range(len(rep_entity_property_pair)):
    tok_sen = tokenizer(dev_ACD_Datas[i], padding='max_length', return_tensors="pt",
                    max_length=256, truncation=True, add_special_tokens=True)  
    
    dev_klue_sets.append(klue_Dataset(tok_sen, dev_ACD_labels[i]))  # klue_Dataset에서 1차원 텐서로 바뀜.
    
dev_klue_sets[24].__getitem__(758)

{'input_ids': tensor([    0,  1187,  2701,  2479,  2113,  2377, 13415,  4137,  2439,  2200,
          3704, 13679,  2965,  3977,  2052,   831,  8974,  2077,  2212,  2319,
         23677,  2088,  2182,     2,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [21]:
test_klue_sets = []

for i in range(len(rep_entity_property_pair)):
    tok_sen = tokenizer(test_ACD_Datas[i], padding='max_length', return_tensors="pt",
                    max_length=256, truncation=True, add_special_tokens=True)  
    
    test_klue_sets.append(klue_Dataset(tok_sen, test_ACD_labels[i]))  # klue_Dataset에서 1차원 텐서로 바뀜.
    
test_klue_sets[24].__getitem__(758)

{'input_ids': tensor([    0,  4482,  2052,  5399, 19661,  2088,  3704,  1943, 19331,  2470,
          4307,  2052,  1637,  1560,  2073,  1345,  1163,  3611,  2073,  5619,
          1363,  2318,  2496,  2259, 16487,     2,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

### Polarity datasets

In [22]:
pol_tok_sen = tokenizer(POL_Datas, padding='max_length', return_tensors="pt",
                    max_length=256, truncation=True, add_special_tokens=True)  

POL_klue_sets = klue_Dataset(pol_tok_sen, POL_labels)

POL_klue_sets.__getitem__(758)

{'input_ids': tensor([    0,  4842,  3797,  1490, 21237,  5021,  1363,  2318,   859,  2259,
          4778,  5478, 16647, 23262,  2996,  2208,  2413,  2100,  2220,  2200,
          2197, 16647, 23262,  2996,  2208,  2413,  2100,  2220,  2667,  2067,
         32003,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [23]:
pol_tok_sen = tokenizer(dev_POL_Datas, padding='max_length', return_tensors="pt",
                    max_length=256, truncation=True, add_special_tokens=True)  

dev_POL_klue_sets = klue_Dataset(pol_tok_sen, dev_POL_labels)

dev_POL_klue_sets.__getitem__(758)

{'input_ids': tensor([    0,  5562,  2975,  2113,  2200,  1126,  2015,  2190,  2356,  2200,
          2119,  1560, 23677,  5515, 32015,     2,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [24]:
pol_tok_sen = tokenizer(test_POL_Datas, padding='max_length', return_tensors="pt",
                    max_length=256, truncation=True, add_special_tokens=True)  

test_POL_klue_sets = klue_Dataset(pol_tok_sen, test_POL_labels)

test_POL_klue_sets.__getitem__(758)

{'input_ids': tensor([    0,  1388,  6233, 12412,  1521,  2144,  2438,  2097, 32003,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

## Trainer 정의

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
    }

In [25]:
for i in range(len(rep_entity_property_pair)):
    
    odir = '/mnt/HDD4T/egg2018037024/ABSA' + str(i)

    training_ars = TrainingArguments(
        output_dir=odir,
        num_train_epochs=5,
     #   max_steps=5000,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        save_total_limit=5,
        save_strategy = "epoch",
        #save_steps=1000,
        learning_rate=5e-5,
        weight_decay=0.01,
        evaluation_strategy='epoch',
     #   load_best_model_at_end = True,
    )

    trainer = Trainer(
        model=ACD_set[i],
        args=training_ars,
        train_dataset=klue_sets[i],
        eval_dataset=dev_klue_sets[i],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    
    trainer.train()
    model.save_pretrained(odir + "_best")

/home/egg2018037024/venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3001
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 470
/home/egg2018037024/venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,0.277897
2,No log,0.293560


***** Running Evaluation *****
  Num examples = 838
  Batch size = 32
Saving model checkpoint to /mnt/HDD4T/egg2018037024/ABSA0/checkpoint-94
Configuration saved in /mnt/HDD4T/egg2018037024/ABSA0/checkpoint-94/config.json
Model weights saved in /mnt/HDD4T/egg2018037024/ABSA0/checkpoint-94/pytorch_model.bin
tokenizer config file saved in /mnt/HDD4T/egg2018037024/ABSA0/checkpoint-94/tokenizer_config.json
Special tokens file saved in /mnt/HDD4T/egg2018037024/ABSA0/checkpoint-94/special_tokens_map.json
/home/egg2018037024/venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 838
  Batch size = 32
Saving model checkpoint to /mnt/HDD4T/egg2018037024/ABSA0/checkpoint-188
Configuration saved in /mnt/HDD4T/egg2018037024/ABSA0/checkpoint-1

RuntimeError: [enforce fail at inline_container.cc:298] . unexpected pos 681660672 vs 681660560